# Porter Stemmer Algorithm
## Natural Language Processing
__Anish Sachdeva__

__DTU/2K16/MC/013__

The Porter Stemmer Algorithm is used very heavily and ubiquitously in the field of Natural Language Processing (NLP) It is used for the morphological analysis of words to find their stems. 

The stem of a word is the main part also referred to as the root.
E.g Happy → happi, dogs → dog, cats → cat, sunny day → sunni day etc.

We find these morphological roots for the tasks of information extraction, information retrieval and also searching data in a big corpus of information.

To impliment the Porter stemmer algorithm and the various steps it follows we first need to impliment a few utility functions.

## Utility Functions
We need to impliment a few utility functions before we can begin implimenting the Porter Stemmer algorithm. These Utility fnctions are as given below.

### Determing Vowels in a Word
We need to determine whether a given letter is a vowel in or a consonant. There are 26 alphabets in the english alphabet out of which 5 are vowels; __a__, __e__, __i__, __o__ and __u__

In [3]:
vowels = ('a', 'e', 'i', 'o', 'u')
def is_vowel(letter):
    return letter in vowels

In [4]:
# test with any alphabet in the following snippet
print(is_vowel('a'))

True


### Determining Consonants in a String
In the Porter stemmer algorith consonants are detected using a special machanism and sometime sdepending on teh context (surrounding leters) __y__ may also be  a vowel. To determing whether an alphabet at index `index` in string `word` is a consonant or not we impliment the following. We also take an integer `start` which denotes the starting index of the word in the string `word` which can represent a big corpora.

In [10]:
    def is_consonant(word, index, start):
        """:returns True if word[index] is a consonant."""
        if is_vowel(word[index]):
            return False
        if word[index] == 'y':
            if index == start:
                return True
            else:
                return not is_consonant(word, index - 1, start)
        return True

In [8]:
print(is_consonant('hello', 4, 0))

False


In [11]:
print(is_consonant('pony', 3, 0))

False


In [13]:
print(is_consonant('test', 2, 0))

True


### Finding the Measure of a Word (m)
In the Porter Stemmer Algorithm, we need to have a measure of the the number of times a Vowel and Consonent sequence occurs. if c is a consonant sequence and v a vowel sequence, and <..> indicates arbitrary presence,

           <c><v>       gives 0
           <c>vc<v>     gives 1
           <c>vcvc<v>   gives 2
           <c>vcvcvc<v> gives 3

we create a method m() which will measure this in a given string.

In [101]:
def m(word):
    n = 0
    i = 0
    offset = len(word) - 1
    start = 0
    while True:
        if i > offset:
            return n
        if not is_consonant(word, i, start):
            break
        i += 1
    i += 1
    while True:
        while True:
            if i > offset:
                return n
            if is_consonant(word, i, start):
                break
            i += 1
        i += 1
        n += 1
        while True:
            if i > offset:
                return n
            if not is_consonant(word, i, start):
                break
            i += 1
        i += 1

In [102]:
for word in ['tree', 'oats', 'trees', 'troubles', 'private']:
    print(word, m(word))

tree 0
oats 1
trees 1
troubles 2
private 2


### Detreming whether the String contains a Vowel or Not
We need to detrmine whether a given string contains a vowel or not and here even a consonant can be treated as a vowel such as __y__ in the range (start, offset). We canwrite this function as follows.

In [98]:
def contains_vowel(word, offset):
    start = 0
    for i in range(start, offset + 1):
        if not is_consonant(word, i, 0):
            return True
    return False

In [99]:
print(contains_vowel('test', 0))

False


In [100]:
print(contains_vowel('test', 2))

True


### Determining Whether the Word Contains a Double Consonant of the Form `pp` `gg` etc. 
We need to determine whether a word contains a double consonant in the word at given index `offset`. 

In [80]:
def contains_double_consonant(word, offset):
    """:returns TRUE if the word contain a double consonant in the range [offset, start]"""
    start = 0
    if offset < (start + 1):
        return False
    if word[offset] != word[offset - 1]:
        return False
    return is_consonant(word, offset, 0)

In [82]:
print(contains_double_consonant('dropped', 4))

True


In [83]:
print(contains_double_consonant('dropped', 3))

False


### Determing Whether a Given String is of Form CVC
Now, we need to define a function that can help is in identifying CVC forms suchthat we have a consonant-vowel-consonant and the consonant is not __x__, __y__ or __z__. e.g. _cav(e)_ , _lov(e)_ , _hop(e)_ , _crim(e)_ etc. and not _snow_ , _box_ , _tray_ ... etc.

In [84]:
    def is_of_form_cvc(word, i):
        start = 0
        if i < (start + 2) or not is_consonant(word, i, 0) or is_consonant(word, i - 1, 0) or not is_consonant(word, i - 2, 0):
            return False
        ch = word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return False
        return True

In [85]:
print(is_of_form_cvc('bat', 2))

True


In [86]:
print(is_of_form_cvc('snow', 3))

False


### Determing Whether a Given String Ends With a Particular String or Not
We will now write a Utility function to determine whether a given sting ends with a particular string or not.

In [123]:
def ends_with(word, s, offset):
    """:returns TRUE when {start...end} ends with the string s."""
    length = len(s)
    start = 0
    end = len(word) - 1
    if s[length - 1] != word[end]:  # tiny speed-up
        return False, offset
    if length > (end - start + 1):
        return False, offset
    if word[end - length + 1 : end + 1] != s:
        return False, offset
    offset = end - length
    return True, offset

In [124]:
print(ends_with('running', 'ing', 0))

(True, 3)


### Utility Function to Set a Suffix/Prefix Stub in a Word
This utility function is of great importance and will be used to set a particular _suffix/prefix_ in the word and this method will be used by the replacement mechanism in the algorithm. 

In [90]:
def set_to(word, s, offset):
    """sets [offset + 1, end] to the characters in the string s, readjusting end."""
    length = len(s)
    word = word[:offset + 1] + s + word[offset + length + 1:]
    return word

In [92]:
print(set_to('happy', 'i', 3))

happi


In [95]:
print(set_to('cats', ' ', 2))

cat 


### Utility Function to  Replace Morphemes in a Word String
This utility function will check the measure `m` of a word an dreplace the morpheme with a given _suffix/prefix_.

In [96]:
def replace_morpheme(word, s, offset):
    """is a mapping function to change morphemes"""
    if m(word) > 0:
        return set_to(word, s, offset)

In [97]:
print(replace_morpheme('happy', 'i', 3))

happi


## Porter Stemmer Algorithm
The porter stemmer algorithm follows the following steps.

## Step 1 A
The following word endings (suffixes) are transformed to other word endings.

SSES → SS

IES → I

SS → SS

S → &epsilon;

## Step 1 B
(m > 0) EED → EE

ED → &epsilon;

ING → &epsilon;

If the second or the third rule are successful then the following need to be done.

AT → ATE

BL → BLE

IZ → IZE

Both these steps are joined in a single function that can be implimented in python as below. The `remove plurals` function will take care of words of the following form:
```text
caresses  ->  caress
ponies    ->  poni
ties      ->  ti
caress    ->  caress
cats      ->  cat

feed      ->  feed
agreed    ->  agree
disabled  ->  disable

matting   ->  mat
mating    ->  mate
meeting   ->  meet
milling   ->  mill
messing   ->  mess

meetings  ->  meet
```
We will create a `PorterStemmer` class that will encapsulate our logic and highly correlated methods and are introducing the `remove_plurals()` method which covers Step 1A and Step 1B.

In [127]:
class PorterStemmer:

    def __init__(self):
        self.vowels = ('a', 'e', 'i', 'o', 'u')
        self.word = ''
        self.end = 0
        self.start = 0
        self.offset = 0

    def is_vowel(self, letter):
        return letter in self.vowels

    def is_consonant(self, index):
        if self.is_vowel(self.word[index]):
            return False
        if self.word[index] == 'y':
            if index == self.start:
                return True
            else:
                return not self.is_consonant(index - 1)
        return True

    def m(self):
        n = 0
        i = self.start
        while True:
            if i > self.offset:
                return n
            if not self.is_consonant(i):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > self.offset:
                    return n
                if self.is_consonant(i):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > self.offset:
                    return n
                if not self.is_consonant(i):
                    break
                i += 1
            i += 1

    def contains_vowel(self):
        for i in range(self.start, self.offset + 1):
            if not self.is_consonant(i):
                return True
        return False

    def contains_double_consonant(self, j):
        if j < (self.start + 1):
            return False
        if self.word[j] != self.word[j - 1]:
            return False
        return self.is_consonant(j)

    def is_of_form_cvc(self, i):
        if i < (self.start + 2) or not self.is_consonant(i) or self.is_consonant(i - 1) or not self.is_consonant(i - 2):
            return 0
        ch = self.word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends_with(self, s):
        length = len(s)
        if s[length - 1] != self.word[self.end]:  # tiny speed-up
            return False
        if length > (self.end - self.start + 1):
            return False
        if self.word[self.end - length + 1: self.end + 1] != s:
            return False
        self.offset = self.end - length
        return True

    def set_to(self, s):
        length = len(s)
        self.word = self.word[:self.offset + 1] + s + self.word[self.offset + length + 1:]
        self.end = self.offset + length

    def replace_morpheme(self, s):
        if self.m() > 0:
            self.set_to(s)

    def remove_plurals(self):
        if self.word[self.end] == 's':
            if self.ends_with("sses"):
                self.end = self.end - 2
            elif self.ends_with("ies"):
                self.set_to("i")
            elif self.word[self.end - 1] != 's':
                self.end = self.end - 1
        if self.ends_with("eed"):
            if self.m() > 0:
                self.end = self.end - 1
        elif (self.ends_with("ed") or self.ends_with("ing")) and self.contains_vowel():
            self.end = self.offset
            if self.ends_with("at"):
                self.set_to("ate")
            elif self.ends_with("bl"):
                self.set_to("ble")
            elif self.ends_with("iz"):
                self.set_to("ize")
            elif self.contains_double_consonant(self.end):
                self.end = self.end - 1
                ch = self.word[self.end]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.end = self.end + 1
            elif self.m() == 1 and self.is_of_form_cvc(self.end):
                self.set_to("e")
                
    def stem_word(self, word):
        if word == '':
            return ''

        self.word = word
        self.end = len(word) - 1
        self.start = 0
        
        # Adding Step 1AB
        self.remove_plurals()
        return self.word[self.start: self.end + 1]

In [128]:
stemmer = PorterStemmer()
print(stemmer.stem_word('caresses'))

caress


### Step 1C
This step turns terminal _y_ to _i_ when there is another vowel in the stem. Adding method `terminal_y_to_i()` in the `PorterStemmer` class.

In [138]:
class PorterStemmer:

    def __init__(self):
        self.vowels = ('a', 'e', 'i', 'o', 'u')
        self.word = ''
        self.end = 0
        self.start = 0
        self.offset = 0

    def is_vowel(self, letter):
        return letter in self.vowels

    def is_consonant(self, index):
        if self.is_vowel(self.word[index]):
            return False
        if self.word[index] == 'y':
            if index == self.start:
                return True
            else:
                return not self.is_consonant(index - 1)
        return True

    def m(self):
        n = 0
        i = self.start
        while True:
            if i > self.offset:
                return n
            if not self.is_consonant(i):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > self.offset:
                    return n
                if self.is_consonant(i):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > self.offset:
                    return n
                if not self.is_consonant(i):
                    break
                i += 1
            i += 1

    def contains_vowel(self):
        for i in range(self.start, self.offset + 1):
            if not self.is_consonant(i):
                return True
        return False

    def contains_double_consonant(self, j):
        if j < (self.start + 1):
            return False
        if self.word[j] != self.word[j - 1]:
            return False
        return self.is_consonant(j)

    def is_of_form_cvc(self, i):
        if i < (self.start + 2) or not self.is_consonant(i) or self.is_consonant(i - 1) or not self.is_consonant(i - 2):
            return 0
        ch = self.word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends_with(self, s):
        length = len(s)
        if s[length - 1] != self.word[self.end]:  # tiny speed-up
            return False
        if length > (self.end - self.start + 1):
            return False
        if self.word[self.end - length + 1: self.end + 1] != s:
            return False
        self.offset = self.end - length
        return True

    def set_to(self, s):
        length = len(s)
        self.word = self.word[:self.offset + 1] + s + self.word[self.offset + length + 1:]
        self.end = self.offset + length

    def replace_morpheme(self, s):
        if self.m() > 0:
            self.set_to(s)

    def remove_plurals(self):
        if self.word[self.end] == 's':
            if self.ends_with("sses"):
                self.end = self.end - 2
            elif self.ends_with("ies"):
                self.set_to("i")
            elif self.word[self.end - 1] != 's':
                self.end = self.end - 1
        if self.ends_with("eed"):
            if self.m() > 0:
                self.end = self.end - 1
        elif (self.ends_with("ed") or self.ends_with("ing")) and self.contains_vowel():
            self.end = self.offset
            if self.ends_with("at"):
                self.set_to("ate")
            elif self.ends_with("bl"):
                self.set_to("ble")
            elif self.ends_with("iz"):
                self.set_to("ize")
            elif self.contains_double_consonant(self.end):
                self.end = self.end - 1
                ch = self.word[self.end]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.end = self.end + 1
            elif self.m() == 1 and self.is_of_form_cvc(self.end):
                self.set_to("e")
                
    def terminal_y_to_i(self):
        if self.ends_with('y') and self.contains_vowel():
            self.word = self.word[:self.end] + 'i' + self.word[self.end + 1:]
                
    def stem_word(self, word):
        if word == '':
            return ''

        self.word = word
        self.end = len(word) - 1
        self.start = 0

        self.remove_plurals()
        ## Adding Step 1C
        self.terminal_y_to_i()
        return self.word[self.start: self.end + 1]

In [133]:
stemmer = PorterStemmer()
print(stemmer.stem_word('happy'))

happi


## Step 2
Defines step 2 and maps double suffices to single ones. so _-ization_ ( = _-ize_ plus _-ation_ ) maps to _-ize_ etc. note that the string before the suffix must give `m` > 0. Implimenting the function `map_double_to_single_suffix` that will take care of the transformations of the following form.
```text
relational --> relate
conditional --> condition
digitizer --> digitize
```

In [139]:
class PorterStemmer:

    def __init__(self):
        self.vowels = ('a', 'e', 'i', 'o', 'u')
        self.word = ''
        self.end = 0
        self.start = 0
        self.offset = 0

    def is_vowel(self, letter):
        return letter in self.vowels

    def is_consonant(self, index):
        if self.is_vowel(self.word[index]):
            return False
        if self.word[index] == 'y':
            if index == self.start:
                return True
            else:
                return not self.is_consonant(index - 1)
        return True

    def m(self):
        n = 0
        i = self.start
        while True:
            if i > self.offset:
                return n
            if not self.is_consonant(i):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > self.offset:
                    return n
                if self.is_consonant(i):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > self.offset:
                    return n
                if not self.is_consonant(i):
                    break
                i += 1
            i += 1

    def contains_vowel(self):
        for i in range(self.start, self.offset + 1):
            if not self.is_consonant(i):
                return True
        return False

    def contains_double_consonant(self, j):
        if j < (self.start + 1):
            return False
        if self.word[j] != self.word[j - 1]:
            return False
        return self.is_consonant(j)

    def is_of_form_cvc(self, i):
        if i < (self.start + 2) or not self.is_consonant(i) or self.is_consonant(i - 1) or not self.is_consonant(i - 2):
            return 0
        ch = self.word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends_with(self, s):
        length = len(s)
        if s[length - 1] != self.word[self.end]:  # tiny speed-up
            return False
        if length > (self.end - self.start + 1):
            return False
        if self.word[self.end - length + 1: self.end + 1] != s:
            return False
        self.offset = self.end - length
        return True

    def set_to(self, s):
        length = len(s)
        self.word = self.word[:self.offset + 1] + s + self.word[self.offset + length + 1:]
        self.end = self.offset + length

    def replace_morpheme(self, s):
        if self.m() > 0:
            self.set_to(s)

    def remove_plurals(self):
        if self.word[self.end] == 's':
            if self.ends_with("sses"):
                self.end = self.end - 2
            elif self.ends_with("ies"):
                self.set_to("i")
            elif self.word[self.end - 1] != 's':
                self.end = self.end - 1
        if self.ends_with("eed"):
            if self.m() > 0:
                self.end = self.end - 1
        elif (self.ends_with("ed") or self.ends_with("ing")) and self.contains_vowel():
            self.end = self.offset
            if self.ends_with("at"):
                self.set_to("ate")
            elif self.ends_with("bl"):
                self.set_to("ble")
            elif self.ends_with("iz"):
                self.set_to("ize")
            elif self.contains_double_consonant(self.end):
                self.end = self.end - 1
                ch = self.word[self.end]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.end = self.end + 1
            elif self.m() == 1 and self.is_of_form_cvc(self.end):
                self.set_to("e")
                
    def terminal_y_to_i(self):
        if self.ends_with('y') and self.contains_vowel():
            self.word = self.word[:self.end] + 'i' + self.word[self.end + 1:]
            
    def map_double_to_single_suffix(self):
        if self.word[self.end - 1] == 'a':
            if self.ends_with("ational"):
                self.replace_morpheme("ate")
            elif self.ends_with("tional"):
                self.replace_morpheme("tion")
        elif self.word[self.end - 1] == 'c':
            if self.ends_with("enci"):
                self.replace_morpheme("ence")
            elif self.ends_with("anci"):
                self.replace_morpheme("ance")
        elif self.word[self.end - 1] == 'e':
            if self.ends_with("izer"):      self.replace_morpheme("ize")
        elif self.word[self.end - 1] == 'l':
            if self.ends_with("bli"):
                self.replace_morpheme("ble")  # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends_with("alli"):
                self.replace_morpheme("al")
            elif self.ends_with("entli"):
                self.replace_morpheme("ent")
            elif self.ends_with("eli"):
                self.replace_morpheme("e")
            elif self.ends_with("ousli"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 'o':
            if self.ends_with("ization"):
                self.replace_morpheme("ize")
            elif self.ends_with("ation"):
                self.replace_morpheme("ate")
            elif self.ends_with("ator"):
                self.replace_morpheme("ate")
        elif self.word[self.end - 1] == 's':
            if self.ends_with("alism"):
                self.replace_morpheme("al")
            elif self.ends_with("iveness"):
                self.replace_morpheme("ive")
            elif self.ends_with("fulness"):
                self.replace_morpheme("ful")
            elif self.ends_with("ousness"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 't':
            if self.ends_with("aliti"):
                self.replace_morpheme("al")
            elif self.ends_with("iviti"):
                self.replace_morpheme("ive")
            elif self.ends_with("biliti"):
                self.replace_morpheme("ble")
        elif self.word[self.end - 1] == 'g':
            if self.ends_with("logi"):      self.replace_morpheme("log")
                
    def stem_word(self, word):
        if word == '':
            return ''

        self.word = word
        self.end = len(word) - 1
        self.start = 0

        self.remove_plurals()
        self.terminal_y_to_i()
        ## adding step 2
        self.map_double_to_single_suffix()
        return self.word[self.start: self.end + 1]

In [140]:
stemmer = PorterStemmer()
print(stemmer.stem_word('digitize'))

digitize


## Step 3
Step 3 deals with -ic-, -full, -ness etc. It leads to the following morphological transformations after implimenting the `step3()` method.
```text
triplicate --> triplic
formative --> form
electrical --> electric
hopeful --> hope
goodness --> good
```

In [142]:
class PorterStemmer:

    def __init__(self):
        self.vowels = ('a', 'e', 'i', 'o', 'u')
        self.word = ''
        self.end = 0
        self.start = 0
        self.offset = 0

    def is_vowel(self, letter):
        return letter in self.vowels

    def is_consonant(self, index):
        if self.is_vowel(self.word[index]):
            return False
        if self.word[index] == 'y':
            if index == self.start:
                return True
            else:
                return not self.is_consonant(index - 1)
        return True

    def m(self):
        n = 0
        i = self.start
        while True:
            if i > self.offset:
                return n
            if not self.is_consonant(i):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > self.offset:
                    return n
                if self.is_consonant(i):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > self.offset:
                    return n
                if not self.is_consonant(i):
                    break
                i += 1
            i += 1

    def contains_vowel(self):
        for i in range(self.start, self.offset + 1):
            if not self.is_consonant(i):
                return True
        return False

    def contains_double_consonant(self, j):
        if j < (self.start + 1):
            return False
        if self.word[j] != self.word[j - 1]:
            return False
        return self.is_consonant(j)

    def is_of_form_cvc(self, i):
        if i < (self.start + 2) or not self.is_consonant(i) or self.is_consonant(i - 1) or not self.is_consonant(i - 2):
            return 0
        ch = self.word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends_with(self, s):
        length = len(s)
        if s[length - 1] != self.word[self.end]:  # tiny speed-up
            return False
        if length > (self.end - self.start + 1):
            return False
        if self.word[self.end - length + 1: self.end + 1] != s:
            return False
        self.offset = self.end - length
        return True

    def set_to(self, s):
        length = len(s)
        self.word = self.word[:self.offset + 1] + s + self.word[self.offset + length + 1:]
        self.end = self.offset + length

    def replace_morpheme(self, s):
        if self.m() > 0:
            self.set_to(s)

    def remove_plurals(self):
        if self.word[self.end] == 's':
            if self.ends_with("sses"):
                self.end = self.end - 2
            elif self.ends_with("ies"):
                self.set_to("i")
            elif self.word[self.end - 1] != 's':
                self.end = self.end - 1
        if self.ends_with("eed"):
            if self.m() > 0:
                self.end = self.end - 1
        elif (self.ends_with("ed") or self.ends_with("ing")) and self.contains_vowel():
            self.end = self.offset
            if self.ends_with("at"):
                self.set_to("ate")
            elif self.ends_with("bl"):
                self.set_to("ble")
            elif self.ends_with("iz"):
                self.set_to("ize")
            elif self.contains_double_consonant(self.end):
                self.end = self.end - 1
                ch = self.word[self.end]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.end = self.end + 1
            elif self.m() == 1 and self.is_of_form_cvc(self.end):
                self.set_to("e")
                
    def terminal_y_to_i(self):
        if self.ends_with('y') and self.contains_vowel():
            self.word = self.word[:self.end] + 'i' + self.word[self.end + 1:]
            
    def map_double_to_single_suffix(self):
        if self.word[self.end - 1] == 'a':
            if self.ends_with("ational"):
                self.replace_morpheme("ate")
            elif self.ends_with("tional"):
                self.replace_morpheme("tion")
        elif self.word[self.end - 1] == 'c':
            if self.ends_with("enci"):
                self.replace_morpheme("ence")
            elif self.ends_with("anci"):
                self.replace_morpheme("ance")
        elif self.word[self.end - 1] == 'e':
            if self.ends_with("izer"):      self.replace_morpheme("ize")
        elif self.word[self.end - 1] == 'l':
            if self.ends_with("bli"):
                self.replace_morpheme("ble")  # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends_with("alli"):
                self.replace_morpheme("al")
            elif self.ends_with("entli"):
                self.replace_morpheme("ent")
            elif self.ends_with("eli"):
                self.replace_morpheme("e")
            elif self.ends_with("ousli"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 'o':
            if self.ends_with("ization"):
                self.replace_morpheme("ize")
            elif self.ends_with("ation"):
                self.replace_morpheme("ate")
            elif self.ends_with("ator"):
                self.replace_morpheme("ate")
        elif self.word[self.end - 1] == 's':
            if self.ends_with("alism"):
                self.replace_morpheme("al")
            elif self.ends_with("iveness"):
                self.replace_morpheme("ive")
            elif self.ends_with("fulness"):
                self.replace_morpheme("ful")
            elif self.ends_with("ousness"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 't':
            if self.ends_with("aliti"):
                self.replace_morpheme("al")
            elif self.ends_with("iviti"):
                self.replace_morpheme("ive")
            elif self.ends_with("biliti"):
                self.replace_morpheme("ble")
        elif self.word[self.end - 1] == 'g':
            if self.ends_with("logi"):      self.replace_morpheme("log")
                
    def step3(self):
        if self.word[self.end] == 'e':
            if self.ends_with("icate"):
                self.replace_morpheme("ic")
            elif self.ends_with("ative"):
                self.replace_morpheme("")
            elif self.ends_with("alize"):
                self.replace_morpheme("al")
        elif self.word[self.end] == 'i':
            if self.ends_with("iciti"):     self.replace_morpheme("ic")
        elif self.word[self.end] == 'l':
            if self.ends_with("ical"):
                self.replace_morpheme("ic")
            elif self.ends_with("ful"):
                self.replace_morpheme("")
        elif self.word[self.end] == 's':
            if self.ends_with("ness"):      self.replace_morpheme("")
                
    def stem_word(self, word):
        if word == '':
            return ''

        self.word = word
        self.end = len(word) - 1
        self.start = 0

        self.remove_plurals()
        self.terminal_y_to_i()
        self.map_double_to_single_suffix()
        # Adding Step 3
        self.step3()
        return self.word[self.start: self.end + 1]

In [150]:
stemmer = PorterStemmer()
print(stemmer.stem_word('goodness'))

good


## Step 4
Step 4 takes off _-ant_ , _-ence_ etc., in context __<c>vcvc<v>__. It will lead to the following morphological transformations:
```text
revival --> reviv
allowance --> allow
inference --> infer
defensible --> defens
```
and many more.

In [151]:
class PorterStemmer:

    def __init__(self):
        self.vowels = ('a', 'e', 'i', 'o', 'u')
        self.word = ''
        self.end = 0
        self.start = 0
        self.offset = 0

    def is_vowel(self, letter):
        return letter in self.vowels

    def is_consonant(self, index):
        if self.is_vowel(self.word[index]):
            return False
        if self.word[index] == 'y':
            if index == self.start:
                return True
            else:
                return not self.is_consonant(index - 1)
        return True

    def m(self):
        n = 0
        i = self.start
        while True:
            if i > self.offset:
                return n
            if not self.is_consonant(i):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > self.offset:
                    return n
                if self.is_consonant(i):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > self.offset:
                    return n
                if not self.is_consonant(i):
                    break
                i += 1
            i += 1

    def contains_vowel(self):
        for i in range(self.start, self.offset + 1):
            if not self.is_consonant(i):
                return True
        return False

    def contains_double_consonant(self, j):
        if j < (self.start + 1):
            return False
        if self.word[j] != self.word[j - 1]:
            return False
        return self.is_consonant(j)

    def is_of_form_cvc(self, i):
        if i < (self.start + 2) or not self.is_consonant(i) or self.is_consonant(i - 1) or not self.is_consonant(i - 2):
            return 0
        ch = self.word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends_with(self, s):
        length = len(s)
        if s[length - 1] != self.word[self.end]:  # tiny speed-up
            return False
        if length > (self.end - self.start + 1):
            return False
        if self.word[self.end - length + 1: self.end + 1] != s:
            return False
        self.offset = self.end - length
        return True

    def set_to(self, s):
        length = len(s)
        self.word = self.word[:self.offset + 1] + s + self.word[self.offset + length + 1:]
        self.end = self.offset + length

    def replace_morpheme(self, s):
        if self.m() > 0:
            self.set_to(s)

    def remove_plurals(self):
        if self.word[self.end] == 's':
            if self.ends_with("sses"):
                self.end = self.end - 2
            elif self.ends_with("ies"):
                self.set_to("i")
            elif self.word[self.end - 1] != 's':
                self.end = self.end - 1
        if self.ends_with("eed"):
            if self.m() > 0:
                self.end = self.end - 1
        elif (self.ends_with("ed") or self.ends_with("ing")) and self.contains_vowel():
            self.end = self.offset
            if self.ends_with("at"):
                self.set_to("ate")
            elif self.ends_with("bl"):
                self.set_to("ble")
            elif self.ends_with("iz"):
                self.set_to("ize")
            elif self.contains_double_consonant(self.end):
                self.end = self.end - 1
                ch = self.word[self.end]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.end = self.end + 1
            elif self.m() == 1 and self.is_of_form_cvc(self.end):
                self.set_to("e")
                
    def terminal_y_to_i(self):
        if self.ends_with('y') and self.contains_vowel():
            self.word = self.word[:self.end] + 'i' + self.word[self.end + 1:]
            
    def map_double_to_single_suffix(self):
        if self.word[self.end - 1] == 'a':
            if self.ends_with("ational"):
                self.replace_morpheme("ate")
            elif self.ends_with("tional"):
                self.replace_morpheme("tion")
        elif self.word[self.end - 1] == 'c':
            if self.ends_with("enci"):
                self.replace_morpheme("ence")
            elif self.ends_with("anci"):
                self.replace_morpheme("ance")
        elif self.word[self.end - 1] == 'e':
            if self.ends_with("izer"):      self.replace_morpheme("ize")
        elif self.word[self.end - 1] == 'l':
            if self.ends_with("bli"):
                self.replace_morpheme("ble")  # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends_with("alli"):
                self.replace_morpheme("al")
            elif self.ends_with("entli"):
                self.replace_morpheme("ent")
            elif self.ends_with("eli"):
                self.replace_morpheme("e")
            elif self.ends_with("ousli"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 'o':
            if self.ends_with("ization"):
                self.replace_morpheme("ize")
            elif self.ends_with("ation"):
                self.replace_morpheme("ate")
            elif self.ends_with("ator"):
                self.replace_morpheme("ate")
        elif self.word[self.end - 1] == 's':
            if self.ends_with("alism"):
                self.replace_morpheme("al")
            elif self.ends_with("iveness"):
                self.replace_morpheme("ive")
            elif self.ends_with("fulness"):
                self.replace_morpheme("ful")
            elif self.ends_with("ousness"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 't':
            if self.ends_with("aliti"):
                self.replace_morpheme("al")
            elif self.ends_with("iviti"):
                self.replace_morpheme("ive")
            elif self.ends_with("biliti"):
                self.replace_morpheme("ble")
        elif self.word[self.end - 1] == 'g':
            if self.ends_with("logi"):      self.replace_morpheme("log")
                
    def step3(self):
        if self.word[self.end] == 'e':
            if self.ends_with("icate"):
                self.replace_morpheme("ic")
            elif self.ends_with("ative"):
                self.replace_morpheme("")
            elif self.ends_with("alize"):
                self.replace_morpheme("al")
        elif self.word[self.end] == 'i':
            if self.ends_with("iciti"):     self.replace_morpheme("ic")
        elif self.word[self.end] == 'l':
            if self.ends_with("ical"):
                self.replace_morpheme("ic")
            elif self.ends_with("ful"):
                self.replace_morpheme("")
        elif self.word[self.end] == 's':
            if self.ends_with("ness"):      self.replace_morpheme("")
                
    def step4(self):
        if self.word[self.end - 1] == 'a':
            if self.ends_with("al"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'c':
            if self.ends_with("ance"):
                pass
            elif self.ends_with("ence"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'e':
            if self.ends_with("er"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'i':
            if self.ends_with("ic"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'l':
            if self.ends_with("able"):
                pass
            elif self.ends_with("ible"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'n':
            if self.ends_with("ant"):
                pass
            elif self.ends_with("ement"):
                pass
            elif self.ends_with("ment"):
                pass
            elif self.ends_with("ent"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'o':
            if self.ends_with("ion") and (self.word[self.offset] == 's' or self.word[self.offset] == 't'):
                pass
            elif self.ends_with("ou"):
                pass
            # takes care of -ous
            else:
                return
        elif self.word[self.end - 1] == 's':
            if self.ends_with("ism"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 't':
            if self.ends_with("ate"):
                pass
            elif self.ends_with("iti"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'u':
            if self.ends_with("ous"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'v':
            if self.ends_with("ive"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'z':
            if self.ends_with("ize"):
                pass
            else:
                return
        else:
            return
        if self.m() > 1:
            self.end = self.offset
                
    def stem_word(self, word):
        if word == '':
            return ''

        self.word = word
        self.end = len(word) - 1
        self.start = 0

        self.remove_plurals()
        self.terminal_y_to_i()
        self.map_double_to_single_suffix()
        self.step3()
        # Adding Step 4 
        self.step4()
        return self.word[self.start: self.end + 1]

In [152]:
stemmer = PorterStemmer()
for word in ['revival', 'allowance', 'inference', 'defensible']:
    print(word, stemmer.stem_word(word))

revival reviv
allowance allow
inference infer
defensible defens


## Step 5
This is the last and final step of the Porter Stemming algorithm after which our algorithm is complete. Step 5 removes a final _-e_ if `m` > 1, and changes __-ll__ to __-l__ if `m` > 1. We will add the method `step5()` and also add 2 API methods `stem_sentence` and `stem_document` to further give the user more ways to stem large corpora. Some examples of morphological changes that are accomplished in this step are:
```text
probate --> probat
rate --> rate
cease --> ceas
controll --> control
roll --> roll
```

In [153]:
class PorterStemmer:

    def __init__(self):
        self.vowels = ('a', 'e', 'i', 'o', 'u')
        self.word = ''
        self.end = 0
        self.start = 0
        self.offset = 0

    def is_vowel(self, letter):
        return letter in self.vowels

    def is_consonant(self, index):
        if self.is_vowel(self.word[index]):
            return False
        if self.word[index] == 'y':
            if index == self.start:
                return True
            else:
                return not self.is_consonant(index - 1)
        return True

    def m(self):
        n = 0
        i = self.start
        while True:
            if i > self.offset:
                return n
            if not self.is_consonant(i):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > self.offset:
                    return n
                if self.is_consonant(i):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > self.offset:
                    return n
                if not self.is_consonant(i):
                    break
                i += 1
            i += 1

    def contains_vowel(self):
        for i in range(self.start, self.offset + 1):
            if not self.is_consonant(i):
                return True
        return False

    def contains_double_consonant(self, j):
        if j < (self.start + 1):
            return False
        if self.word[j] != self.word[j - 1]:
            return False
        return self.is_consonant(j)

    def is_of_form_cvc(self, i):
        if i < (self.start + 2) or not self.is_consonant(i) or self.is_consonant(i - 1) or not self.is_consonant(i - 2):
            return 0
        ch = self.word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends_with(self, s):
        length = len(s)
        if s[length - 1] != self.word[self.end]:  # tiny speed-up
            return False
        if length > (self.end - self.start + 1):
            return False
        if self.word[self.end - length + 1: self.end + 1] != s:
            return False
        self.offset = self.end - length
        return True

    def set_to(self, s):
        length = len(s)
        self.word = self.word[:self.offset + 1] + s + self.word[self.offset + length + 1:]
        self.end = self.offset + length

    def replace_morpheme(self, s):
        if self.m() > 0:
            self.set_to(s)

    def remove_plurals(self):
        if self.word[self.end] == 's':
            if self.ends_with("sses"):
                self.end = self.end - 2
            elif self.ends_with("ies"):
                self.set_to("i")
            elif self.word[self.end - 1] != 's':
                self.end = self.end - 1
        if self.ends_with("eed"):
            if self.m() > 0:
                self.end = self.end - 1
        elif (self.ends_with("ed") or self.ends_with("ing")) and self.contains_vowel():
            self.end = self.offset
            if self.ends_with("at"):
                self.set_to("ate")
            elif self.ends_with("bl"):
                self.set_to("ble")
            elif self.ends_with("iz"):
                self.set_to("ize")
            elif self.contains_double_consonant(self.end):
                self.end = self.end - 1
                ch = self.word[self.end]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.end = self.end + 1
            elif self.m() == 1 and self.is_of_form_cvc(self.end):
                self.set_to("e")
                
    def terminal_y_to_i(self):
        if self.ends_with('y') and self.contains_vowel():
            self.word = self.word[:self.end] + 'i' + self.word[self.end + 1:]
            
    def map_double_to_single_suffix(self):
        if self.word[self.end - 1] == 'a':
            if self.ends_with("ational"):
                self.replace_morpheme("ate")
            elif self.ends_with("tional"):
                self.replace_morpheme("tion")
        elif self.word[self.end - 1] == 'c':
            if self.ends_with("enci"):
                self.replace_morpheme("ence")
            elif self.ends_with("anci"):
                self.replace_morpheme("ance")
        elif self.word[self.end - 1] == 'e':
            if self.ends_with("izer"):      self.replace_morpheme("ize")
        elif self.word[self.end - 1] == 'l':
            if self.ends_with("bli"):
                self.replace_morpheme("ble")  # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends_with("alli"):
                self.replace_morpheme("al")
            elif self.ends_with("entli"):
                self.replace_morpheme("ent")
            elif self.ends_with("eli"):
                self.replace_morpheme("e")
            elif self.ends_with("ousli"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 'o':
            if self.ends_with("ization"):
                self.replace_morpheme("ize")
            elif self.ends_with("ation"):
                self.replace_morpheme("ate")
            elif self.ends_with("ator"):
                self.replace_morpheme("ate")
        elif self.word[self.end - 1] == 's':
            if self.ends_with("alism"):
                self.replace_morpheme("al")
            elif self.ends_with("iveness"):
                self.replace_morpheme("ive")
            elif self.ends_with("fulness"):
                self.replace_morpheme("ful")
            elif self.ends_with("ousness"):
                self.replace_morpheme("ous")
        elif self.word[self.end - 1] == 't':
            if self.ends_with("aliti"):
                self.replace_morpheme("al")
            elif self.ends_with("iviti"):
                self.replace_morpheme("ive")
            elif self.ends_with("biliti"):
                self.replace_morpheme("ble")
        elif self.word[self.end - 1] == 'g':
            if self.ends_with("logi"):      self.replace_morpheme("log")
                
    def step3(self):
        if self.word[self.end] == 'e':
            if self.ends_with("icate"):
                self.replace_morpheme("ic")
            elif self.ends_with("ative"):
                self.replace_morpheme("")
            elif self.ends_with("alize"):
                self.replace_morpheme("al")
        elif self.word[self.end] == 'i':
            if self.ends_with("iciti"):     self.replace_morpheme("ic")
        elif self.word[self.end] == 'l':
            if self.ends_with("ical"):
                self.replace_morpheme("ic")
            elif self.ends_with("ful"):
                self.replace_morpheme("")
        elif self.word[self.end] == 's':
            if self.ends_with("ness"):      self.replace_morpheme("")
                
    def step4(self):
        if self.word[self.end - 1] == 'a':
            if self.ends_with("al"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'c':
            if self.ends_with("ance"):
                pass
            elif self.ends_with("ence"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'e':
            if self.ends_with("er"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'i':
            if self.ends_with("ic"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'l':
            if self.ends_with("able"):
                pass
            elif self.ends_with("ible"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'n':
            if self.ends_with("ant"):
                pass
            elif self.ends_with("ement"):
                pass
            elif self.ends_with("ment"):
                pass
            elif self.ends_with("ent"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'o':
            if self.ends_with("ion") and (self.word[self.offset] == 's' or self.word[self.offset] == 't'):
                pass
            elif self.ends_with("ou"):
                pass
            # takes care of -ous
            else:
                return
        elif self.word[self.end - 1] == 's':
            if self.ends_with("ism"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 't':
            if self.ends_with("ate"):
                pass
            elif self.ends_with("iti"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'u':
            if self.ends_with("ous"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'v':
            if self.ends_with("ive"):
                pass
            else:
                return
        elif self.word[self.end - 1] == 'z':
            if self.ends_with("ize"):
                pass
            else:
                return
        else:
            return
        if self.m() > 1:
            self.end = self.offset
            
    def step5(self):
        self.offset = self.end
        if self.word[self.end] == 'e':
            a = self.m()
            if a > 1 or (a == 1 and not self.is_of_form_cvc(self.end - 1)):
                self.end = self.end - 1
        if self.word[self.end] == 'l' and self.contains_double_consonant(self.end) and self.m() > 1:
            self.end = self.end - 1
                
    def stem_document(self, document):
        result = []
        for line in document.split('\n'):
            result.append(self.stem_sentence(line))
        return '\n'.join(result)
    
    def stem_sentence(self, sentence):
        result = []
        for word in sentence.split():
            result.append(self.stem_word(word))
        return ' '.join(result)
    
    def stem_word(self, word):
        if word == '':
            return ''

        self.word = word
        self.end = len(word) - 1
        self.start = 0

        self.remove_plurals()
        self.terminal_y_to_i()
        self.map_double_to_single_suffix()
        self.step3()
        self.step4()
        # adding step 5
        self.step5()
        return self.word[self.start: self.end + 1]

In [156]:
stemmer = PorterStemmer()
for word in ['controll', 'roll', 'cease', 'probate']:
    print(word, stemmer.stem_word(word))

controll control
roll roll
cease ceas
probate probat


## Seeing the Stemming in Action
Now, we have successfully implimented the Poretr stemmer algorithm, you can test any valid english string and see the stemmed output, just modify the `test_string` below.

In [159]:
stemmer = PorterStemmer()
test_string = 'stones and sticks may break my bones but words can never hurt me'
print(stemmer.stem_sentence(test_string))

stone and stick mai break my bone but word can never hurt me


# Running the Porter Stemmer on Resume/CV
The porter stemmer also has the `stem_document` API and can be used on large corpora of textual information. Let us first load our resume and see it's contents. 

In [157]:
resume = open('resume.txt', 'r').read()
print(resume)

Anish Sachdeva
Software Developer + Clean Code Enthusiast

Phone : 8287428181
email : anish_@outlook.com
home : sandesh vihar, pitampura, new delhi - 110034
date of birth : 7th April 1998
languages : English, Hindi, French

Work Experience
What After College (4 months)
Delhi, India
Creating content to teach Core Java and Python with Data Structures and Algorithms and giving online classes to students

Summer Research Fellow at University of Auckland (2 Months)
Auckland, New Zealand
Worked on Geometry of Mobius Transformations, Differential Grometry under Dr. Pedram Hekmati at the Department of Mathematics, University of Auckland

Software Developer at CERN (14 Months)
CERN, Geneva, Switzerland
Worked in the core Platforms team of the FAP-BC group. Part of an agile team of developers that maintains and adds core functionality to applications used internally at CERN by HR, Financial, Administrative and other departments including Scientific
Worked on legacy applications that comprise of 

Now, we will pass this string document into our stemmer and see the stemmed output. 

In [160]:
stemmer = PorterStemmer()
stemmed_resume = stemmer.stem_document(resume)
print(stemmed_resume)

Anish Sachdeva
Softwar Develop + Clean Code Enthusiast

Phone : 8287428181
email : anish_@outlook.com
home : sandesh vihar, pitampura, new delhi - 110034
date of birth : 7th April 1998
languag : English, Hindi, French

Work Experienc
What After Colleg (4 months)
Delhi, India
Creat content to teach Core Java and Python with Data Structur and Algorithm and give onlin class to student

Summer Research Fellow at Univers of Auckland (2 Months)
Auckland, New Zealand
Work on Geometri of Mobiu Transformations, Differenti Grometri under Dr. Pedram Hekmati at the Depart of Mathematics, Univers of Auckland

Softwar Develop at CERN (14 Months)
CERN, Geneva, Switzerland
Work in the core Platform team of the FAP-BC group. Part of an agil team of develop that maintain and add core function to applic us intern at CERN by HR, Financial, Administr and other depart includ Scientif
Work on legaci applic that compris of singl and some time multipl framework such a Java Spring, Boot, Hibern and Java EE. Als

# Analytics on Resume and Stemmed Output
We will create 2 utility functions.
1. To calculate the number of words in a document
2. To calculate Unique words in a document

## Number of Words In A Document (Word Counter)

In [161]:
def number_of_words(document):
    count = 0
    for line in document.split('\n'):
        count += len(line.split())
    return count

In [162]:
number_words_resume = number_of_words(resume)
number_words_stemmed_resume = number_of_words(stemmed_resume)

## Number of Unique Words in Document

In [163]:
def unique_words(document):
    words = set()
    for line in document.split('\n'):
        for word in line.split():
            words.add(word)
    return len(words)

In [164]:
number_unique_words_resume = unique_words(resume)
number_unique_words_stemmed_resume = unique_words(stemmed_resume)

In [167]:
print('The number of words originally in the resume were:', number_words_resume, 'words')
print('After Stemming the document had:', number_words_stemmed_resume, ' words')
print('The unique words in the resume were:', number_unique_words_resume, 'words')
print('The unique words after stemming the resume were:', number_unique_words_stemmed_resume, 'words')
print('So total reduction in unique words after stemming was:', number_unique_words_resume - number_unique_words_stemmed_resume, 'words')

The number of words originally in the resume were: 369 words
After Stemming the document had: 369  words
The unique words in the resume were: 251 words
The unique words after stemming the resume were: 241 words
So total reduction in unique words after stemming was: 10 words
